In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.imports import *

In [3]:
PATH = '/home/paperspace/data/materialist/'
sz=224
arch=resnext50
#arch=resnet34
bs=32

In [4]:
label_csv = f'{PATH}labels_valid.csv'
label_df = pd.read_csv(label_csv)

In [5]:
n = len(list(open(label_csv))) -1 
val_idxs = get_cv_idxs(n, val_pct=.21)

In [6]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, f'{PATH}valid', f'{PATH}labels_valid.csv', test_name='test', 
                                    val_idxs=val_idxs, 
                                    tfms=tfms, 
                                    bs=bs)

In [7]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=.5)

In [10]:
learn.save('materialist_valid')

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 1e-3
learn.fit(lr, 2, cycle_len=1)

In [ ]:
learn.save('materialist_50_1_precompture')

In [9]:
#learn.freeze()
learn.precompute = False

In [ ]:
learn.fit(lr, 3, cycle_len=1)

In [9]:
learn.precompute = False
learn.load('materialist_50_2')

In [12]:
learn.unfreeze()

In [ ]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))









  1%|▏         | 2/156 [00:01<02:17,  1.12it/s, loss=2.75]

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss   accuracy                   
    0      2.175945   1.744142   0.552598  
 71%|███████   | 111/156 [00:54<00:22,  2.04it/s, loss=1.71]

In [ ]:
learn.save('materialist_50_2_unfrozen')

In [ ]:
def get_data(sz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    return ImageClassifierData.from_csv(PATH, f'{PATH}merged/train', f'{PATH}labels_2.csv', test_name='test', 
                                    val_idxs=val_idxs, 
                                    tfms=tfms, 
                                    bs=bs)

In [ ]:
learn.set_data(get_data(299))

In [ ]:
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:
learn.save('materialist_299')

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.load('materialist_299_2')

In [ ]:
log_preds,y = learn.TTA(n_aug=4, is_test=True)
preds = np.mean(log_preds, 0)
probs = np.mean(np.exp(log_preds), axis=0)
indices = probs.argmax(axis=1)

In [ ]:
accuracy_np(probs, y)

In [ ]:
preds = probs.argmax(axis=1)

In [ ]:
indexed_preds = [data.classes[pred] for pred in preds]

In [ ]:
def trim_jpeg(filename):
    slash_index = filename.index('/') + 1
    dot_index = filename.index('.')
    return filename[slash_index:dot_index]

In [ ]:
def get_filename_indices():
    filenames = data.test_ds.fnames
    filenames = [int(trim_jpeg(filename)) for filename in filenames]
    return filenames

In [ ]:
def get_filenames():
    filenames = data.test_ds.fnames
    filenames = [trim_jpeg(filename) for filename in filenames]
    return filenames

In [ ]:
def get_missing(indices):
    acc = []
    for x in range(1, 12800):
        if x not in indices:
            acc.append(x)
    return acc

In [ ]:
missing_indices = get_missing(get_filename_indices())

In [ ]:
filenames = get_filenames()

In [ ]:
missing_indices

In [ ]:
with open(f'{PATH}large_submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(('id', 'predicted'))
    writer.writerows(zip(filenames, indexed_preds))
    for m in missing_indices:
        writer.writerow((m, 1))

In [ ]:
def get_missing(indices):
    acc = []
    for x in range(1, 12800):
        if x not in indices:
            acc.append(x)
    return acc

In [ ]:
def make_submission(preds, incidces):
    missing = get_missing(indices)
    with open(f'{PATH}submission.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(('id', 'predicted'))
        writer.writerows(zip(indices, preds))
        for m in missing:
            writer.writerow((m, 1))

In [ ]:
make_submission(indexed_preds, indices)